In [1]:
print("Hello Wold!")

Hello Wold!


In [2]:
# !pip install pymongo
# !pip install plotly
# !pip install pandas
# !pip install nbformat

In [3]:
import plotly.graph_objs as go
import pandas as pd
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["SteelArena"]
collection = db["Event_Kabat"]

In [4]:
def create_mongo_pipeline_with_timezone_adjustment(minutes, field_prefix):
    # Determine the value field based on the prefix ('t' for 'T' sensors, 's' for 'S' sensors)
    value_field = 't' if field_prefix.startswith('T') else 's'

    pipeline = [
        {
            '$project': {
                '_id': 1,
                field_prefix: 1,  # Dynamically use the field prefix here
                'time': {
                    '$toDate': '$_id'  # Convert ObjectId to date
                }
            }
        },
        {
            '$addFields': {
                'kosice_time': {
                    '$dateAdd': {
                        'startDate': '$time',
                        'unit': 'hour',
                        'amount': 2  # Add 2 hours for Košice timezone (UTC+2)
                    }
                }
            }
        },
        {
            '$match': {
                f'{field_prefix}.{value_field}': {
                    '$exists': True  # Match only documents where the specific sensor data exists
                }
            }
        },
        {
            '$group': {
                '_id': {
                    '$dateTrunc': {
                        'date': '$kosice_time',
                        'unit': 'minute',
                        'binSize': minutes  # Dynamically set the bin size here
                    }
                },
                f'{field_prefix}_values': {
                    # Push the sensor values (either 't' or 's')
                    '$push': f'${field_prefix}.{value_field}'
                }
            }
        },
        {
            '$project': {
                '_id': 1,
                f'median_{field_prefix}': {
                    '$let': {
                        'vars': {
                            'sorted_values': {
                                '$sortArray': {
                                    'input': f'${field_prefix}_values',
                                    'sortBy': 1
                                }
                            }
                        },
                        'in': {
                            '$arrayElemAt': [
                                '$$sorted_values',
                                {
                                    '$floor': {
                                        '$divide': [
                                            {
                                                '$size': '$$sorted_values'
                                            }, 2
                                        ]
                                    }
                                }
                            ]
                        }
                    }
                }
            }
        },
        {
            '$sort': {
                '_id': -1  # Sorting by _id (time) in descending order
            }
        }
    ]
    return pipeline

In [5]:
def create_sensor_dataframe(collection, sensors, minutes):
    # Initialize an empty dictionary to store the sensor data
    sensor_data = {'time': []}

    # Iterate over each sensor in the list
    for sensor in sensors:
        # Run the pipeline for the current sensor
        pipeline = create_mongo_pipeline_with_timezone_adjustment(
            minutes, sensor)
        data = list(collection.aggregate(pipeline))

        # Extract time data only once (for the first sensor)
        if 'time' not in sensor_data or len(sensor_data['time']) == 0:
            sensor_data['time'] = [item['_id']
                                   # The time field is stored in _id
                                   for item in data]

        # Extract sensor values and add them to the dictionary
        sensor_data[sensor] = [
            item[f'median_{sensor}'] if isinstance(item[f'median_{sensor}'], dict)
            else item[f'median_{sensor}'] for item in data
        ]

    # Create a DataFrame from the sensor data dictionary
    df_sensors = pd.DataFrame(sensor_data)

    return df_sensors

In [6]:
def create_sensor_dataframe(collection, sensors, minutes):
    # Initialize an empty dictionary to store the sensor data
    sensor_data = {}

    # Iterate over each sensor in the list
    for sensor in sensors:
        # Run the pipeline for the current sensor
        pipeline = create_mongo_pipeline_with_timezone_adjustment(
            minutes, sensor)
        data = list(collection.aggregate(pipeline))

        # Create a temporary DataFrame for the current sensor
        df_temp = pd.DataFrame({
            'time': [item['_id'] for item in data],
            sensor: [item[f'median_{sensor}'] if isinstance(item[f'median_{sensor}'], dict)
                     else item[f'median_{sensor}'] for item in data]
        })

        # Merge the sensor data into the main DataFrame
        if 'time' not in sensor_data:
            sensor_data = df_temp  # Initialize with the first sensor data
        else:
            # Merge based on 'time'
            sensor_data = pd.merge(sensor_data, df_temp,
                                   on='time', how='outer')

    # Return the final DataFrame, which includes data from all sensors
    return sensor_data

In [7]:
# Example usage
sensors = ['T6', 'T5', 'T4', 'T3', 'T2', 'T1', "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
           "S15", "S16", "S17", "S18", "S19", "S20", "S21", "S22", "S23", "S24", "S25", "S26", "S27", "S28", "S29"]

minutes = 10 

# Assuming `collection` is your MongoDB collection
df_sensors = create_sensor_dataframe(collection, sensors, minutes)

In [8]:
def tare_s_sensors_by_nearest_6am(df_sensors):
    # Identify S sensor columns (columns that start with 'S')
    s_sensor_columns = [
        col for col in df_sensors.columns if col.startswith('S')]

    # Define the target time (6:00 AM) as a Timedelta
    target_time = pd.Timedelta(hours=6)

    # Tare each S sensor column based on the nearest value to 6 AM
    for s_sensor in s_sensor_columns:
        # Calculate the absolute difference from 6 AM using full datetime
        df_sensors['time_diff'] = abs(
            df_sensors['time'] - (df_sensors['time'].dt.normalize() + target_time))

        # Find the index of the row closest to 6 AM
        nearest_time_index = df_sensors['time_diff'].idxmin()

        # Get the sensor value at the nearest time to 6 AM
        tare_value = df_sensors.loc[nearest_time_index, s_sensor]

        # Subtract the tare value from the entire column
        df_sensors[s_sensor] = df_sensors[s_sensor] - tare_value

    # Drop the temporary 'time_diff' column
    df_sensors = df_sensors.drop(columns=['time_diff'])

    return df_sensors


def tare_and_multiply_s_sensors(df_sensors, s_multiplier=3.11e-9*210e3):
    # Identify S sensor columns (columns that start with 'S')
    s_sensor_columns = [
        col for col in df_sensors.columns if col.startswith('S')]

    # Subtract the tare value from each column and multiply by the s_multiplier
    for s_sensor in s_sensor_columns:
        # The values are already tared in the previous step
        df_sensors[s_sensor] = df_sensors[s_sensor] * s_multiplier

    return df_sensors

In [9]:
# Assuming df_sensors is your DataFrame created by `create_sensor_dataframe`
df_sensors = tare_s_sensors_by_nearest_6am(
    df_sensors)  # Tare based on the nearest 6 AM value
# Multiply the tared values by s_multiplier
df_sensors = tare_and_multiply_s_sensors(df_sensors)

In [10]:
def plot_s_and_t_sensors(df_sensors, s_sensor, t_sensor, title="Koncert"):
    fig = go.Figure()

    # Extract time data
    time_data = df_sensors['time']

    # Plot S sensor values on the left y-axis
    fig.add_trace(go.Scatter(
        x=time_data,
        y=df_sensors[s_sensor],
        mode='lines',
        name=f"{s_sensor} Napätia",
        yaxis='y1'  # Left y-axis
    ))

    # Plot T sensor values on the right y-axis
    fig.add_trace(go.Scatter(
        x=time_data,
        y=df_sensors[t_sensor],
        mode='lines',
        name=f"{t_sensor} Teploty",
        yaxis='y2'  # Right y-axis
    ))

    # Update layout to add secondary y-axis and format the plot
    fig.update_layout(
        title=title,
        # xaxis_title="Time",
        yaxis=dict(
            title=f"{s_sensor} Napätia [MPa]",  # Left y-axis title
            side="left"
        ),
        yaxis2=dict(
            title=f"{t_sensor} Teploty [°C]",  # Right y-axis title
            overlaying='y',  # Overlay on the same plot
            side='right',
            showgrid=False  # Turn off grid for the temperature axis
        ),
        legend=dict(
            orientation="h",  # Horizontal legend
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template="plotly"
    )

    # Show the plot
    fig.show()

In [11]:
plot_s_and_t_sensors(df_sensors, s_sensor='S27', t_sensor='T4',
                     title="Dolná tribúna, 15. október 24, koncert Kabát (lokálny čas)")
plot_s_and_t_sensors(df_sensors, s_sensor='S29', t_sensor='T6',
                     title="Horná tribúna, 15. október 24, koncert Kabát (lokálny čas)")
plot_s_and_t_sensors(df_sensors, s_sensor='S28', t_sensor='T5',
                     title="Stĺp, 15. október 24, koncert Kabát (lokálny čas)")

In [12]:
def plot_s_sensor_pairs(df_sensors, sensor_pairs, title="Tenzometrické páry, 15. október 24, koncert Kabát (lokálny čas)"):
    fig = go.Figure()

    # Get the time column
    time_data = df_sensors['time']

    # Plot each pair of S sensors
    for s_sensor in sensor_pairs:
        if s_sensor in df_sensors.columns:
            fig.add_trace(go.Scatter(
                x=time_data,
                y=df_sensors[s_sensor],
                mode='lines',
                name=s_sensor
            ))

    # Add titles and labels
    fig.update_layout(
        title=title,
        # xaxis_title="Time",
        yaxis_title="Napätia [MPa]",
        template="plotly",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Show the plot
    fig.show()

In [13]:
# # Assuming df_sensors is your DataFrame
# sensor_pairs = ["S13", "S25"]  # List of specific sensor pairs you want to plot
# plot_s_sensor_pairs(df_sensors, sensor_pairs,
#                     title="Tenzometrické páry, Rez 2-4 (horné), 15. október 24, Kabát (lokálny čas)")

# sensor_pairs = ["S11", "S22"]  # List of specific sensor pairs you want to plot
# plot_s_sensor_pairs(df_sensors, sensor_pairs,
#                     title="Tenzometrické páry, Rez 2-4 (diagonály), 15. október 24, Kabát (lokálny čas)")

# sensor_pairs = ["S14", "S26"]  # List of specific sensor pairs you want to plot
# plot_s_sensor_pairs(df_sensors, sensor_pairs,
#                     title="Tenzometrické páry, Rez 2-4 (spodné ľavé), 15. október 24, Kabát (lokálny čas)")

# sensor_pairs = ["S12", "S23"]  # List of specific sensor pairs you want to plot
# plot_s_sensor_pairs(df_sensors, sensor_pairs,
#                     title="Tenzometrické páry, Rez 2-4 (spodné pravé), 15. október 24, Kabát (lokálny čas)")

In [14]:
# # Example usage:
# # Assuming df_sensors is your DataFrame
# sensor_pairs = ["S7", "S21"]  # List of specific sensor pairs you want to plot
# plot_s_sensor_pairs(df_sensors, sensor_pairs,
#                     title="Tenzometrické páry, Rez 1-5 (horné), 15. október 24, Kabát (lokálny čas)")

# sensor_pairs = ["S8", "S24"]  # List of specific sensor pairs you want to plot
# plot_s_sensor_pairs(df_sensors, sensor_pairs,
#                     title="Tenzometrické páry, Rez 1-5 (diagonály), 15. október 24, Kabát (lokálny čas)")

# sensor_pairs = ["S10", "S19"]  # List of specific sensor pairs you want to plot
# plot_s_sensor_pairs(df_sensors, sensor_pairs,
#                     title="Tenzometrické páry, Rez 1-5 (spodné ľavé), 15. október 24, Kabát (lokálny čas)")
# sensor_pairs = ["S9", "S20"]  # List of specific sensor pairs you want to plot
# plot_s_sensor_pairs(df_sensors, sensor_pairs,
#                     title="Tenzometrické páry, Rez 1-5 (spodné pravé), 15. október 24, Kabát (lokálny čas)")

In [15]:
def create_pipeline(sensor_name):
    pipeline = [
        {
            '$match': {
                f'{sensor_name}.fft.spectrum': {
                    '$exists': True,
                    '$ne': None
                }
            }
        },
        {
            '$project': {
                f'{sensor_name}.fft.spectrum': 1,
                'time': 1
            }
        },
        {
            '$sort': {
                'time.datetime': 1
            }
        }
    ]
    return pipeline

In [16]:
# Example usage for pulling data for sensor "A31"
sensor_name = "A33"
pipeline = create_pipeline(sensor_name)

# Run the pipeline to get the data
result = list(collection.aggregate(pipeline))

# Transform the data into the required format
data = []
for doc in result:
    time = doc['time']['datetime']
    spectrum = doc[sensor_name]['fft']['spectrum']

    # Prepare the row where first element is time, followed by spectrum as individual dictionaries
    row = {'time': time}
    for idx, entry in enumerate(spectrum):
        row[idx] = {'f': entry['f'], 'a': entry['a']}

    data.append(row)

# Create DataFrame where each column corresponds to an index in the spectrum list
df = pd.DataFrame(data)

# Display the DataFrame
df.shape

(4320, 31)

In [17]:
# Check if there are any NaN values in the entire DataFrame
nan_check = df.isna().any().any()

if nan_check:
    print("There are NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")

There are no NaN values in the DataFrame.


In [18]:
import plotly.graph_objs as go
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
from IPython.display import clear_output, display

# Define a function to update the plot based on the row index and number of previous spectrums


def update_plot(row_index, previous_spectrums):
    # Clear the previous output
    clear_output(wait=True)

    # Create the scatter plot
    fig = go.Figure()

    # Plot the previous spectrums if possible
    for i in range(1, previous_spectrums + 1):
        if row_index - i >= 0:
            prev_row = df.iloc[row_index - i]
            prev_frequencies = []
            prev_amplitudes = []
            for j in range(30):  # Assuming there are 30 spectrum columns (0-29)
                spectrum_data = prev_row[j]
                prev_frequencies.append(spectrum_data['f'])
                prev_amplitudes.append(spectrum_data['a'])

            # Add the previous spectrum data as a scatter plot with smaller markers and alpha
            fig.add_trace(go.Scatter(
                x=prev_frequencies,
                y=prev_amplitudes,
                mode='markers',
                # Same color, smaller markers with alpha
                marker=dict(size=4, opacity=0.7, color='blue'),
                name=f'Previous Spectrum - Row {row_index - i}'
            ))

    # Extract the frequencies and amplitudes from the specified row
    frequencies = []
    amplitudes = []

    row = df.iloc[row_index]
    for i in range(30):  # Assuming there are 30 spectrum columns (0-29)
        spectrum_data = row[i]
        frequencies.append(spectrum_data['f'])
        amplitudes.append(spectrum_data['a'])

    # Get the 'time' value from the row
    time_value = row['time']

    # Add the current spectrum data as a scatter plot
    fig.add_trace(go.Scatter(
        x=frequencies,
        y=amplitudes,
        mode='markers',
        marker=dict(size=6, color='blue'),  # Same color for current markers
        name=f'Spectrum - Row {row_index}'
    ))

    # Update the layout of the plot
    fig.update_layout(
        title=f"Frequency vs Amplitude Scatter Plot",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Amplitude",
        showlegend=False,
        xaxis=dict(range=[0, 100]),  # Set x-axis limits from 0 to 100
        yaxis=dict(range=[0, 1.5])
    )

    # Add the 'time' value in the bottom-right corner
    fig.add_annotation(
        xref="paper", yref="paper",
        x=1, y=-0.125,  # Position at the bottom-right
        text=f"{time_value}",
        showarrow=False,
        font=dict(size=12),
        xanchor='right',
        yanchor='bottom'
    )

    # Show the plot
    display(fig)


# Create a slider widget for selecting the row index
row_slider = widgets.IntSlider(value=2691, min=0, max=len(
    df) - 1, step=1, description='Row Index:', layout=widgets.Layout(width='90%'))

# Create a text input widget for selecting the number of previous spectrums to plot
previous_spectrums_input = widgets.BoundedIntText(
    value=100, min=0, max=2000, step=1, description='Previous Spectrums:')

# Use the interact function to update the plot whenever the inputs change
interact(update_plot, row_index=row_slider,
         previous_spectrums=previous_spectrums_input)

interactive(children=(IntSlider(value=2691, description='Row Index:', layout=Layout(width='90%'), max=4319), B…

<function __main__.update_plot(row_index, previous_spectrums)>

In [21]:
df.iloc[0]

time                             2024-10-14T22:00:19
0         {'f': 16.6015625, 'a': 0.1368490308523178}
1          {'f': 16.40625, 'a': 0.06298521906137466}
2        {'f': 16.796875, 'a': 0.060572680085897446}
3          {'f': 99.21875, 'a': 0.05327841639518738}
4         {'f': 5.6640625, 'a': 0.05145706981420517}
5       {'f': 34.1796875, 'a': 0.050188906490802765}
6         {'f': 9.9609375, 'a': 0.04779969900846481}
7           {'f': 2.34375, 'a': 0.04565194249153137}
8         {'f': 17.578125, 'a': 0.04487568140029907}
9           {'f': 47.65625, 'a': 0.0417909100651741}
10       {'f': 66.6015625, 'a': 0.04156113415956497}
11       {'f': 78.3203125, 'a': 0.04119054600596428}
12        {'f': 4.8828125, 'a': 0.04115383327007294}
13       {'f': 28.7109375, 'a': 0.03946902975440025}
14       {'f': 23.046875, 'a': 0.039062947034835815}
15         {'f': 8.203125, 'a': 0.03891885280609131}
16       {'f': 91.9921875, 'a': 0.03871980309486389}
17          {'f': 60.9375, 'a': 0.037790983915